# About this example

This example shows how you can deploy ALBERT model to analyze confidential text for sentiment analysis. The model will be left untrained for demonstration purposes. We could finetune it on positive/negative samples before deploying it.

By using BlindAI, people can send data for the AI to analyze sensitive text without having to fear privacy leaks.

ALBERT is a state of the art Transformers model for NLP. You can learn more about it on this [Hugging Face page](https://huggingface.co/docs/transformers/model_doc/albert).

More information on this use case can be found on our blog post [Deploy Transformers models with confidentiality](https://blog.mithrilsecurity.io/transformers-with-confidentiality/).

# Installing dependencies

Install the dependencies this example needs.

In [ ]:
!pip install transformers[onnx] torch

Install the latest version of BlindAI.

In [ ]:
!pip install blindai

# Preparing the model

In this first step we will export a standard Hugging Face ALBERT model to an ONNX file, as BlindAI accepts only ONNX files.

In [ ]:
import os
import torch
from transformers import AutoModelForMaskedLM, AutoTokenizer

model_name = "albert-base-v2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

text = "Paris is the [MASK] of France."
tokenizer_output = tokenizer(text, return_tensors="pt")

input_ids = tokenizer_output["input_ids"]
attention_mask = tokenizer_output["attention_mask"]
token_type_ids = tokenizer_output["token_type_ids"]

dynamic_axes = {
    0: "batch",
    1: "seq",
}

output_dir = "./albert"
os.makedirs(output_dir, exist_ok=True)
torch.onnx.export(
    model,
    (input_ids, attention_mask, token_type_ids),
    os.path.join(output_dir, "model.onnx"),
    input_names=["input_ids", "attention_mask", "token_type_ids"],
    output_names=["logits"],
    dynamic_axes={
        "input_ids": dynamic_axes,
        "attention_mask": dynamic_axes,
        "token_type_ids": dynamic_axes,
        "logits": dynamic_axes,
    },
    opset_version=13,
)

tokenizer.save_pretrained(output_dir)

# Deployment on BlindAI

In [ ]:
import blindai

# Launch client
with blindai.connect() as client:
    response = client.upload_model(
        model="albert/model.onnx"
    )
    model_id = response.model_id

This securely uploads the model to the Mithril Cloud.
If you wish to run this example on premise, you should read the [Deploy on Hardware](https://blindai.mithrilsecurity.io/en/latest/getting-started/deploy-on-hardware/) documentation page. 

# Sending data for confidential prediction

Now it's time to check it's working live!

We will just prepare the inputs for the model inside the secure enclave of BlindAI to process it.

In [ ]:
inputs = tokenizer(text, return_tensors="pt")["input_ids"]
token_type = tokenizer(text, return_tensors="pt")["token_type_ids"]
attention = tokenizer(text, return_tensors="pt")["attention_mask"]

Now we can get a prediction.

In [ ]:
import blindai
import numpy as np

with blindai.connect() as client:
  response = client.predict(model_id, [inputs, token_type, attention])
print(response.output[0].as_flat()[:20])

Here we can compare the results against the original prediction.

In [ ]:

outputs = model(inputs, token_type, attention).logits.detach()

print(outputs[0][0].tolist()[:20])


Et voila! We have been able to apply a start of the art model of NLP, without ever having to show the data in clear to the people operating the service!

If you have liked this example, do not hesitate to drop a star on our [GitHub](https://github.com/mithril-security/blindai) and chat with us on our [Discord](https://discord.gg/TxEHagpWd4)!